In [93]:
import os, glob, json, requests, re, shutil


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from PIL import Image


In [94]:
url_to = 'https://online.fliphtml5.com/iequa/kkdd/'

In [95]:
def process_browser_log_entry(entry):
    response = json.loads(entry['message'])['message']
    return response

In [96]:
options = webdriver.ChromeOptions()
options.add_argument('w3c')
options.add_argument('headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--window-size=1920,1080')
options.add_argument('--start-maximized')

caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options.set_capability('cloud:options', caps)

service = ChromeService(ChromeDriverManager().install())
DRIVER = webdriver.Chrome(service=service, options=options)

In [97]:
DRIVER.get(url_to)

In [98]:
browser_log = DRIVER.get_log('performance') 
events = [process_browser_log_entry(entry) for entry in browser_log]
events = [event for event in events if 'Network.response' in event['method']]

In [99]:
url_config = ''
for event in events:
    if 'params' in event and 'response' in event['params'] and 'url' in event['params']['response'] and 'config.js' in event['params']['response']['url']:
        print(event['params']['response']['url'])
        url_config = event['params']['response']['url']

https://online.fliphtml5.com/iequa/kkdd/javascript/config.js?1675846573


In [100]:
r = requests.get(url_config)

In [101]:
data_json = r.text.replace('var htmlConfig = ', '').replace(';', '')

In [102]:
data_json = json.loads(data_json)

In [103]:
image_ls = data_json['fliphtml5_pages']

In [104]:
#image_ls

In [105]:
image_base_url = data_json['meta']['url'].replace('index.html', '') + data_json['bookConfig']['largePath']

In [106]:
image_base_url

'https://online.fliphtml5.com/iequa/kkdd/files/large/'

In [107]:
filename = data_json['meta']['title']
filename = re.sub('[^0-9a-zA-Z]+', '_', filename) + '.pdf'

In [108]:
filename

'Bahan_Tayang_Stress_Management_Chapter_4_Avoid_.pdf'

In [109]:
if not os.path.exists('fliphtml5_temp'):
    os.makedirs('fliphtml5_temp')

for f in os.listdir('fliphtml5_temp'):
    os.remove(os.path.join('fliphtml5_temp', f))

In [112]:
if len(image_ls) > 0:
    i = 0
    images = []
    for image in image_ls:
        if isinstance(image[list(image.keys())[0]], list):
            original_image_name = image[list(image.keys())[0]][0].replace('files/large/', '')
        else:
            original_image_name = image[list(image.keys())[0]].replace('files/large/', '')
        image_url = image_base_url + original_image_name 
        print(i, image_url)
        r = requests.get(image_url, stream=True)
        if r.status_code == 200:
            with open('fliphtml5_temp/' + str(i) + '.' + original_image_name.split('.')[-1], 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
                images.append(Image.open('fliphtml5_temp/' + str(i) + '.' + original_image_name.split('.')[-1]))

            i += 1
    
    images[0].save(filename, save_all=True, append_images=images[1:])

https://online.fliphtml5.com/iequa/kkdd/files/large/5e52b4184b1022f4a1b99a27c8d0aa12.jpg
https://online.fliphtml5.com/iequa/kkdd/files/large/37693cfc748049e45d87b8c7d8b9aacd.jpg
https://online.fliphtml5.com/iequa/kkdd/files/large/e6cdbdf8135f776cbc6f962f20056c7f.jpg
https://online.fliphtml5.com/iequa/kkdd/files/large/d9d4f495e875a2e075a1a4a6e1b9770f.jpg
https://online.fliphtml5.com/iequa/kkdd/files/large/c467a926202625be53163e200463443e.jpg
https://online.fliphtml5.com/iequa/kkdd/files/large/d0b1bfd50dd40176f497a2915a6e579b.jpg
https://online.fliphtml5.com/iequa/kkdd/files/large/50a3edb92c326be64e1f2ccd2553aca8.jpg
https://online.fliphtml5.com/iequa/kkdd/files/large/98511f963d72e390b80aee100dda064d.jpg
https://online.fliphtml5.com/iequa/kkdd/files/large/f46beb405b2a3b907775da6fc7aa2289.jpg
https://online.fliphtml5.com/iequa/kkdd/files/large/6d0b462c813d6fff0cda410c17a54440.jpg


In [ ]:
DRIVER.quit()

In [ ]:
image_ls

[{'n': ['5e52b4184b1022f4a1b99a27c8d0aa12.jpg'],
  't': './files/thumb/856bd39b46518a2837c9b6121601c415.jpg'},
 {'n': ['37693cfc748049e45d87b8c7d8b9aacd.jpg'],
  't': './files/thumb/7831fc6814a693477f41c070f218d3de.jpg'},
 {'n': ['e6cdbdf8135f776cbc6f962f20056c7f.jpg'],
  't': './files/thumb/dcfe0b3ef8eb8156a5e83949c2e2f7b6.jpg'},
 {'n': ['d9d4f495e875a2e075a1a4a6e1b9770f.jpg'],
  't': './files/thumb/aea1e33f16dc583f59f3742eeee96b03.jpg'},
 {'n': ['c467a926202625be53163e200463443e.jpg'],
  't': './files/thumb/e7f1750ff9b954647a71e50caa31bda7.jpg'},
 {'n': ['d0b1bfd50dd40176f497a2915a6e579b.jpg'],
  't': './files/thumb/d0b1bfd50dd40176f497a2915a6e579b.jpg'},
 {'n': ['50a3edb92c326be64e1f2ccd2553aca8.jpg'],
  't': './files/thumb/b11c3f22bffb354ac5305cb1f6477fa4.jpg'},
 {'n': ['98511f963d72e390b80aee100dda064d.jpg'],
  't': './files/thumb/43ec517d68b6edd3015b3edc9a11367b.jpg'},
 {'n': ['f46beb405b2a3b907775da6fc7aa2289.jpg'],
  't': './files/thumb/f78518fb11533620004f6cfd93aebee6.jpg'},
 